# Email File Attachment System Demonstration

© Crown-owned copyright 2025, Defence Science and Technology Laboratory UK

This notebook demonstrates the complete email file attachment system in PrimAITE-Mail, including:

- Basic attachment sending and receiving
- Policy enforcement and security features
- Malware scanning and threat detection
- Multi-agent scenarios with red/blue/green team interactions
- Performance testing and troubleshooting examples

## Overview

The email attachment system enables realistic file transfer scenarios through email:

| **Feature** | **Purpose** | **Security Implications** |
|:------------|:------------|:-------------------------|
| **File Attachments** | Realistic email workflows | Malware distribution vector |
| **Policy Enforcement** | Size and type restrictions | Prevent dangerous files |
| **Health Preservation** | Maintain file corruption status | Simulate malware persistence |
| **Malware Scanning** | Threat detection | Identify suspicious files |
| **Agent Actions** | Red/Blue/Green team scenarios | Cybersecurity training |
| **Quarantine System** | Threat response | Defensive measures |

## Learning Objectives

1. **Basic Operations**: Send and receive emails with attachments
2. **Security Features**: Policy enforcement and threat detection
3. **Malware Scanning**: Understand threat classification
4. **Agent Integration**: Multi-agent cybersecurity scenarios
5. **Performance**: Handle large attachments and multiple files
6. **Troubleshooting**: Debug common attachment issues

## Table of Contents

- [Environment Setup](#environment-setup)
- [Basic Attachment Operations](#basic-attachment-operations)
- [Policy Enforcement and Security](#policy-enforcement-and-security)
- [Malware Scanning Demonstration](#malware-scanning-demonstration)
- [Multi-Agent Security Scenarios](#multi-agent-security-scenarios)
- [Performance Testing](#performance-testing)
- [Troubleshooting Guide](#troubleshooting-guide)

## Environment Setup

Initialize the PrimAITE-Mail environment and import required components.

In [ ]:
# Setup PrimAITE environment
!primaite setup

In [ ]:
# Import required modules
import primaite
from primaite import PRIMAITE_CONFIG
from primaite.simulator.network.container import Network
from primaite.simulator.network.hardware.nodes.host.computer import Computer
from primaite.simulator.file_system.file_system import FileSystem
from primaite.simulator.file_system.file_type import FileType
from primaite.simulator.file_system.file_system_item_abc import FileSystemItemHealthStatus

from primaite_mail.simulator.software.smtp_server import SMTPServer
from primaite_mail.simulator.software.pop3_server import POP3Server
from primaite_mail.simulator.software.email_client import EmailClient
from primaite_mail.simulator.network.protocols.smtp import EmailMessage
from primaite_mail.simulator.network.protocols.email_attachments import EmailAttachment, AttachmentPolicy
from primaite_mail.simulator.network.protocols.attachment_manager import AttachmentManager

# Store original config to restore later
original_dev_mode = PRIMAITE_CONFIG["developer_mode"]["enabled"]
original_sys_logs = PRIMAITE_CONFIG["developer_mode"]["output_sys_logs"]
original_agent_logs = PRIMAITE_CONFIG["developer_mode"]["output_agent_logs"]

# Enable developer mode and system logs for debugging
PRIMAITE_CONFIG["developer_mode"]["enabled"] = True
PRIMAITE_CONFIG["developer_mode"]["output_sys_logs"] = True
PRIMAITE_CONFIG["developer_mode"]["output_agent_logs"] = True

print("Email attachment system demonstration initialized")
print(f"Developer mode: {PRIMAITE_CONFIG['developer_mode']['enabled']}")

## Network and Infrastructure Setup

Create a network with mail server and multiple clients for attachment testing.

In [ ]:
def create_attachment_network():
    """Create network infrastructure for attachment demonstration."""
    network = Network()
    
    # Create mail server
    mail_server = Computer.from_config({
        "type": "computer",
        "hostname": "mail_server",
        "ip_address": "192.168.1.10",
        "subnet_mask": "255.255.255.0",
        "start_up_duration": 0,
    })
    mail_server.power_on()
    network.add_node(mail_server)
    
    # Create client computers
    client_configs = [
        ("alice_pc", "192.168.1.20"),   # Green team - legitimate user
        ("bob_pc", "192.168.1.21"),     # Green team - legitimate user
        ("red_pc", "192.168.1.30"),     # Red team - attacker
        ("blue_pc", "192.168.1.40"),    # Blue team - defender
    ]
    
    clients = {}
    for hostname, ip in client_configs:
        client = Computer.from_config({
            "type": "computer",
            "hostname": hostname,
            "ip_address": ip,
            "subnet_mask": "255.255.255.0",
            "start_up_duration": 0,
        })
        client.power_on()
        network.add_node(client)
        clients[hostname] = client
    
    return network, clients

# Create the network
print("Creating attachment demonstration network...")
network, clients = create_attachment_network()
mail_server = network.get_node_by_hostname("mail_server")

print(f"Network created with {len(network.nodes)} nodes:")
for hostname in ["mail_server", "alice_pc", "bob_pc", "red_pc", "blue_pc"]:
    node = network.get_node_by_hostname(hostname)
    print(f"  {hostname}: {node.config.ip_address}")

## Email Services Installation and Configuration

In [ ]:
# Install email servers on mail server
print("Installing email services...")
mail_server.software_manager.install(SMTPServer)
mail_server.software_manager.install(POP3Server)

smtp_server = mail_server.software_manager.software.get("smtp-server")
pop3_server = mail_server.software_manager.software.get("pop3-server")

# Configure attachment policy with security settings
attachment_policy = AttachmentPolicy(
    max_attachment_size=10 * 1024 * 1024,  # 10MB per attachment
    max_total_size=25 * 1024 * 1024,       # 25MB total message size
    max_attachments=5,                      # Maximum 5 attachments per email
    allowed_extensions=["txt", "pdf", "doc", "docx", "xls", "xlsx", "jpg", "png"],
    blocked_extensions=["exe", "bat", "scr", "com", "pif"],
    scan_for_malware=True,
    quarantine_suspicious=True
)

# Start SMTP server with attachment policy
smtp_server.start()

# Share mailbox manager and start POP3
pop3_server.mailbox_manager = smtp_server.mailbox_manager
pop3_server.start()

print(f"SMTP Server: {smtp_server.name} (Port: {smtp_server.port})")
print(f"POP3 Server: {pop3_server.name} (Port: {pop3_server.port})")
print(f"Attachment Policy: Max size {attachment_policy.max_attachment_size // (1024*1024)}MB")
print(f"Allowed extensions: {attachment_policy.allowed_extensions}")
print(f"Blocked extensions: {attachment_policy.blocked_extensions}")

In [ ]:
# Install email clients and create mailboxes
print("Setting up email clients and mailboxes...")

email_clients = {}
users = ["alice", "bob", "red_agent", "blue_agent"]

# Map users to their computers
user_computers = {
    "alice": clients["alice_pc"],
    "bob": clients["bob_pc"], 
    "red_agent": clients["red_pc"],
    "blue_agent": clients["blue_pc"]
}

for user in users:
    # Create mailbox on server
    smtp_server.mailbox_manager.create_mailbox(user)
    
    # Install and configure email client
    computer = user_computers[user]
    computer.software_manager.install(EmailClient)
    client = computer.software_manager.software.get("email-client")
    
    # Configure client
    client.config.username = f"{user}@company.com"
    client.config.default_smtp_server = str(mail_server.config.ip_address)
    client.config.default_pop3_server = str(mail_server.config.ip_address)
    
    email_clients[user] = client
    print(f"  {user}: {client.name} on {computer.config.hostname}")

print(f"\nCreated {len(users)} mailboxes and email clients")

## Create Test Files for Attachments

Create various types of files to demonstrate different attachment scenarios.

In [ ]:
def create_test_files():
    """Create test files on different computers for attachment scenarios."""
    
    # Alice's legitimate business documents
    alice_pc = clients["alice_pc"]
    alice_fs = alice_pc.file_system
    
    # Create documents using file system methods
    alice_fs.create_folder("documents")
    
    # Business documents
    report_file = alice_fs.create_file(
        folder_name="documents",
        file_name="quarterly_report.pdf",
        file_type=FileType.PDF,
        size=2048  # 2KB
    )
    
    spreadsheet_file = alice_fs.create_file(
        folder_name="documents",
        file_name="budget.xlsx",
        file_type=FileType.XLSX,
        size=1536  # 1.5KB
    )
    
    # Red agent's malicious files
    red_pc = clients["red_pc"]
    red_fs = red_pc.file_system
    
    # Create malware folder
    red_fs.create_folder("payloads")
    
    # Corrupted executable (simulated malware)
    malware_file = red_fs.create_file(
        folder_name="payloads",
        file_name="update.exe",
        file_type=FileType.PE,  # Executable
        size=4096  # 4KB
    )
    # Mark as corrupted to simulate malware
    malware_file.health_status = FileSystemItemHealthStatus.CORRUPT
    
    # Disguised malware as document
    trojan_file = red_fs.create_file(
        folder_name="payloads",
        file_name="invoice.pdf",
        file_type=FileType.PDF,
        size=3072  # 3KB
    )
    # Mark as corrupted to simulate malware
    trojan_file.health_status = FileSystemItemHealthStatus.CORRUPT
    
    # Double extension attack file
    double_ext_file = red_fs.create_file(
        folder_name="payloads",
        file_name="document.pdf.exe",
        file_type=FileType.PE,
        size=2048
    )
    
    # Suspicious batch file
    batch_file = red_fs.create_file(
        folder_name="payloads",
        file_name="script.bat",
        file_type=FileType.BAT,
        size=1024
    )
    
    # Bob's files for collaboration
    bob_pc = clients["bob_pc"]
    bob_fs = bob_pc.file_system
    
    # Create shared folder
    bob_fs.create_folder("shared")
    
    # Presentation file
    presentation_file = bob_fs.create_file(
        folder_name="shared",
        file_name="project_update.pptx",
        file_type=FileType.PPTX,
        size=5120  # 5KB
    )
    
    return {
        "alice_docs": [("documents", "quarterly_report.pdf"), ("documents", "budget.xlsx")],
        "red_malware": [("payloads", "update.exe"), ("payloads", "invoice.pdf"), ("payloads", "document.pdf.exe"), ("payloads", "script.bat")],
        "bob_shared": [("shared", "project_update.pptx")]
    }

# Create test files
print("Creating test files for attachment scenarios...")
test_files = create_test_files()

print("\nTest files created:")
for category, files in test_files.items():
    print(f"  {category}: {len(files)} files")
    for folder, filename in files:
        print(f"    - {folder}/{filename}")

# Display file system status
print("\n=== FILE SYSTEM STATUS ===")
for name, computer in [("Alice", clients["alice_pc"]), ("Red Agent", clients["red_pc"]), ("Bob", clients["bob_pc"])]:
    print(f"\n{name}'s Files:")
    computer.file_system.show()

## Basic Attachment Operations

Demonstrate basic email attachment sending and receiving using the AttachmentManager.

In [ ]:
# Initialize AttachmentManager
attachment_manager = AttachmentManager()

print("=== BASIC ATTACHMENT OPERATIONS ===")
print("\n1. Creating attachments from files...")

# Alice creates attachments from her files
alice_pc = clients["alice_pc"]
alice_fs = alice_pc.file_system

# Attach the quarterly report
report_attachment = attachment_manager.attach_file(
    file_system=alice_fs,
    folder_name="documents",
    file_name="quarterly_report.pdf"
)

# Attach the budget spreadsheet
budget_attachment = attachment_manager.attach_file(
    file_system=alice_fs,
    folder_name="documents",
    file_name="budget.xlsx"
)

if report_attachment and budget_attachment:
    print(f"✓ Created attachment: {report_attachment.filename} ({report_attachment.file_size} bytes)")
    print(f"✓ Created attachment: {budget_attachment.filename} ({budget_attachment.file_size} bytes)")
    
    # Create email with attachments
    business_email = EmailMessage(
        sender="alice@company.com",
        recipients=["bob@company.com"],
        subject="Q4 Reports and Budget",
        body="Hi Bob,\n\nPlease find attached the Q4 financial report and budget spreadsheet for review.\n\nBest regards,\nAlice"
    )
    
    # Add attachments to email
    business_email.add_attachment(report_attachment)
    business_email.add_attachment(budget_attachment)
    
    print(f"\n2. Email created with {business_email.attachment_count} attachments")
    print(f"   Total email size: {business_email.calculate_total_size()} bytes")
    print(f"   Has attachments: {business_email.has_attachments}")
    
else:
    print("✗ Failed to create attachments")

In [ ]:
# Send email through SMTP server
print("\n3. Sending email with attachments...")

# Use network request to send email
response = network.apply_request([
    "node", "mail_server", "service", "smtp-server", "send_email", 
    {
        "email_data": business_email.model_dump(),
        "sender_ip": str(clients["alice_pc"].config.ip_address)
    }
])

if response.status == "success":
    print("✓ Email sent successfully")
    print(f"   Response: {response.data}")
else:
    print(f"✗ Email sending failed: {response.data}")

# Show server status after sending
print("\n=== SERVER STATUS AFTER SENDING ===")
smtp_server.show()
smtp_server.show_mailbox("bob")

In [ ]:
# Bob retrieves and extracts attachments
print("\n4. Retrieving and extracting attachments...")

# Get Bob's mailbox
bob_mailbox = smtp_server.mailbox_manager.get_mailbox("bob")
if bob_mailbox:
    messages = bob_mailbox.get_messages()
    print(f"Bob has {len(messages)} messages")
    
    if messages:
        # Get the first message
        message = messages[0]
        print(f"\nMessage from: {message.sender}")
        print(f"Subject: {message.subject}")
        print(f"Attachments: {message.attachment_count}")
        
        # Extract attachments to Bob's file system
        bob_pc = clients["bob_pc"]
        bob_fs = bob_pc.file_system
        
        # Create downloads folder
        bob_fs.create_folder("downloads")
        
        # Extract each attachment
        for attachment in message.attachments:
            success, error = attachment_manager.extract_attachment(
                attachment=attachment,
                file_system=bob_fs,
                destination_folder="downloads"
            )
            
            if success:
                print(f"✓ Extracted: {attachment.filename}")
            else:
                print(f"✗ Failed to extract {attachment.filename}: {error}")
        
        # Show Bob's downloads folder
        print("\n=== BOB'S DOWNLOADS FOLDER ===")
        downloads_folder = bob_fs.get_folder("downloads")
        if downloads_folder:
            for filename, file_obj in downloads_folder.files.items():
                if not file_obj.deleted:
                    print(f"  {filename}: {file_obj.size} bytes, {file_obj.health_status.name}")
else:
    print("✗ Bob's mailbox not found")

## Policy Enforcement and Security

Demonstrate attachment policy enforcement and security features.

In [ ]:
print("=== ATTACHMENT POLICY ENFORCEMENT ===")

# Test policy validation
print("\n1. Testing attachment policy validation...")

# Create a test policy
test_policy = AttachmentPolicy(
    max_attachment_size=1024,  # 1KB limit (very small for testing)
    max_total_size=2048,       # 2KB total
    max_attachments=2,
    allowed_extensions=["txt", "pdf"],
    blocked_extensions=["exe", "bat"],
    scan_for_malware=True
)

print(f"Policy: Max {test_policy.max_attachment_size} bytes per file")
print(f"Allowed: {test_policy.allowed_extensions}")
print(f"Blocked: {test_policy.blocked_extensions}")

# Test with Alice's legitimate files
if 'report_attachment' in locals() and report_attachment:
    is_valid, error = attachment_manager.validate_attachment(report_attachment, test_policy)
    print(f"\nReport PDF validation: {'✓ PASS' if is_valid else '✗ FAIL'}")
    if not is_valid:
        print(f"  Reason: {error}")

# Test extension checking
print("\n2. Testing extension policies...")
test_extensions = ["document.pdf", "script.exe", "data.txt", "malware.bat"]

for filename in test_extensions:
    allowed = test_policy.is_extension_allowed(filename)
    print(f"  {filename}: {'✓ ALLOWED' if allowed else '✗ BLOCKED'}")

## Malware Scanning Demonstration

Demonstrate the malware scanning capabilities and threat classification.

In [ ]:
print("=== MALWARE SCANNING DEMONSTRATION ===")

# Create various test attachments for scanning
red_pc = clients["red_pc"]
red_fs = red_pc.file_system

print("\n1. Creating test attachments with different threat levels...")

# Create attachments from different files
test_attachments = []

# Clean document
if 'report_attachment' in locals() and report_attachment:
    test_attachments.append(report_attachment)
    print(f"✓ Clean file: {report_attachment.filename}")

# Create malicious attachments
malicious_files = [
    ("payloads", "update.exe", "Malware executable"),
    ("payloads", "invoice.pdf", "Disguised malware"),
    ("payloads", "document.pdf.exe", "Double extension attack"),
    ("payloads", "script.bat", "Suspicious batch file")
]

for folder, filename, description in malicious_files:
    attachment = attachment_manager.attach_file(
        file_system=red_fs,
        folder_name=folder,
        file_name=filename
    )
    if attachment:
        test_attachments.append(attachment)
        print(f"✓ {description}: {attachment.filename} (Health: {attachment.health_status})")

print(f"\nTotal test attachments: {len(test_attachments)}")

In [ ]:
# Perform malware scanning
print("\n2. Performing malware scan...")

if test_attachments:
    scan_results = attachment_manager.scan_for_malware(test_attachments)
    
    print("\nScan Results:")
    print("-" * 60)
    for filename, result in scan_results.items():
        status_icon = {
            "CLEAN": "✓",
            "MALWARE_DETECTED": "🚨",
            "SUSPICIOUS_EXECUTABLE": "⚠️",
            "SUSPICIOUS_EXTENSION": "⚠️",
            "SUSPICIOUS_DOUBLE_EXTENSION": "🚨",
            "SUSPICIOUS_SIZE": "⚠️",
            "SCAN_ERROR": "❌"
        }.get(result, "?")
        
        print(f"{status_icon} {filename:<25} -> {result}")
    
    # Analyze results
    print("\n3. Threat Analysis:")
    threat_counts = {}
    for result in scan_results.values():
        threat_counts[result] = threat_counts.get(result, 0) + 1
    
    for threat_type, count in threat_counts.items():
        print(f"  {threat_type}: {count} files")
    
    # Security recommendations
    print("\n4. Security Recommendations:")
    if "MALWARE_DETECTED" in threat_counts:
        print("  🚨 CRITICAL: Malware detected - quarantine immediately")
    if "SUSPICIOUS_DOUBLE_EXTENSION" in threat_counts:
        print("  🚨 HIGH RISK: Double extension attack detected")
    if "SUSPICIOUS_EXECUTABLE" in threat_counts or "SUSPICIOUS_EXTENSION" in threat_counts:
        print("  ⚠️  MEDIUM RISK: Suspicious executable files detected")
    if "CLEAN" in threat_counts:
        print(f"  ✓ {threat_counts['CLEAN']} files appear safe")
        
    # Explain threat classifications
    print("\n5. Threat Classification Guide:")
    print("  MALWARE_DETECTED: File health status is CORRUPT")
    print("  SUSPICIOUS_DOUBLE_EXTENSION: Files like 'document.pdf.exe'")
    print("  SUSPICIOUS_EXTENSION: Files with dangerous extensions (.exe, .bat, etc.)")
    print("  SUSPICIOUS_EXECUTABLE: Files with executable MIME types")
    print("  SUSPICIOUS_SIZE: Files larger than 100MB")
    print("  CLEAN: Files that pass all security checks")
else:
    print("No test attachments available for scanning")

## Multi-Agent Security Scenarios

Demonstrate red team attacks and blue team defensive responses.

In [ ]:
print("=== MULTI-AGENT SECURITY SCENARIO ===")
print("\nScenario: Red team attempts to send malware via email")

# Red team creates malicious email
print("\n1. Red Team Attack - Creating malicious email...")

# Get malicious attachments
malware_attachment = None
trojan_attachment = None

for attachment in test_attachments:
    if attachment.filename == "update.exe":
        malware_attachment = attachment
    elif attachment.filename == "invoice.pdf":
        trojan_attachment = attachment

if malware_attachment and trojan_attachment:
    # Create malicious email
    malicious_email = EmailMessage(
        sender="red_agent@company.com",
        recipients=["alice@company.com", "bob@company.com"],
        subject="Urgent: System Update Required",
        body="Dear Team,\n\nPlease install the attached security update immediately. Also find the invoice for recent purchases.\n\nIT Department"
    )
    
    # Add malicious attachments
    malicious_email.add_attachment(malware_attachment)
    malicious_email.add_attachment(trojan_attachment)
    
    print(f"Red team email created:")
    print(f"  Subject: {malicious_email.subject}")
    print(f"  Recipients: {malicious_email.recipients}")
    print(f"  Attachments: {malicious_email.attachment_count}")
    
    # Attempt to send through server
    print("\n2. Server Security Response...")
    
    # Validate attachments against policy
    policy_results = []
    for attachment in malicious_email.attachments:
        is_valid, error = attachment_manager.validate_attachment(attachment, attachment_policy)
        policy_results.append((attachment.filename, is_valid, error))
        
        if not is_valid:
            print(f"  🚨 BLOCKED: {attachment.filename} - {error}")
        else:
            print(f"  ⚠️  PASSED POLICY: {attachment.filename}")
    
    # Perform malware scan
    scan_results = attachment_manager.scan_for_malware(malicious_email.attachments)
    print("\n3. Malware Scan Results:")
    for filename, result in scan_results.items():
        if result == "MALWARE_DETECTED":
            print(f"  🚨 MALWARE: {filename} - QUARANTINED")
        elif result.startswith("SUSPICIOUS"):
            print(f"  ⚠️  SUSPICIOUS: {filename} - {result}")
        else:
            print(f"  ✓ CLEAN: {filename}")
    
    # Log security violations
    print("\n4. Security Logging...")
    for attachment in malicious_email.attachments:
        is_valid, error = attachment_manager.validate_attachment(attachment, attachment_policy)
        if not is_valid:
            attachment_manager.log_policy_violation(
                attachment=attachment,
                policy=attachment_policy,
                violation_reason=error,
                sender=malicious_email.sender,
                recipients=malicious_email.recipients
            )
            print(f"  📝 Logged violation: {attachment.filename}")
else:
    print("Malicious attachments not available for scenario")

In [ ]:
# Blue team defensive analysis
print("\n5. Blue Team Analysis...")

# Analyze threat patterns
print("\nThreat Intelligence:")
print(f"  - Attack vector: Email attachments")
print(f"  - Threat actor: red_agent@company.com")
print(f"  - Social engineering: Urgent system update theme")
print(f"  - File types: Executable (.exe) and disguised PDF")
print(f"  - Health status: CORRUPT (indicating malware)")

# Defensive recommendations
print("\nDefensive Recommendations:")
print("  1. 🛡️  Block all executable attachments (.exe, .bat, .scr)")
print("  2. 🔍 Enable deep content inspection for PDFs")
print("  3. 📧 Implement sender reputation checking")
print("  4. 🎓 Conduct user awareness training on social engineering")
print("  5. 🔒 Quarantine suspicious attachments automatically")
print("  6. 📊 Monitor for similar attack patterns")

# System hardening
print("\nSystem Hardening Applied:")
hardened_policy = AttachmentPolicy(
    max_attachment_size=5 * 1024 * 1024,  # 5MB
    max_total_size=10 * 1024 * 1024,      # 10MB
    max_attachments=3,
    allowed_extensions=["txt", "pdf", "doc", "docx", "xls", "xlsx", "jpg", "png"],
    blocked_extensions=["exe", "bat", "cmd", "scr", "pif", "com", "vbs", "js"],
    scan_for_malware=True,
    quarantine_suspicious=True
)

print(f"  - Reduced max attachment size to {hardened_policy.max_attachment_size // (1024*1024)}MB")
print(f"  - Expanded blocked extensions: {hardened_policy.blocked_extensions}")
print(f"  - Enabled automatic quarantine: {hardened_policy.quarantine_suspicious}")
print(f"  - Enhanced malware scanning: {hardened_policy.scan_for_malware}")

## Performance Testing

Test attachment system performance with various file sizes and types.

In [ ]:
import time

print("=== PERFORMANCE TESTING ===")

# Create performance test files
print("\n1. Creating performance test files...")

alice_fs = clients["alice_pc"].file_system
alice_fs.create_folder("perf_test")

# Test different file sizes
test_sizes = [
    ("small.txt", FileType.TXT, 1024),        # 1KB
    ("medium.pdf", FileType.PDF, 100*1024),   # 100KB
    ("large.xlsx", FileType.XLSX, 1024*1024), # 1MB
]

perf_files = []
for filename, file_type, size in test_sizes:
    file_obj = alice_fs.create_file(
        folder_name="perf_test",
        file_name=filename,
        file_type=file_type,
        size=size
    )
    perf_files.append((filename, size))
    print(f"  Created {filename}: {size:,} bytes")

# Performance benchmarks
print("\n2. Performance Benchmarks:")
print("-" * 60)
print(f"{'File':<15} {'Size':<10} {'Attach Time':<12} {'Scan Time':<10} {'Status'}")
print("-" * 60)

for filename, size in perf_files:
    # Time attachment creation
    start_time = time.time()
    attachment = attachment_manager.attach_file(
        file_system=alice_fs,
        folder_name="perf_test",
        file_name=filename
    )
    attach_time = time.time() - start_time
    
    if attachment:
        # Time malware scanning
        start_time = time.time()
        scan_results = attachment_manager.scan_for_malware([attachment])
        scan_time = time.time() - start_time
        
        scan_result = scan_results.get(filename, "UNKNOWN")
        
        print(f"{filename:<15} {size:>8,}B {attach_time:>8.3f}s {scan_time:>8.3f}s {scan_result}")
    else:
        print(f"{filename:<15} {size:>8,}B {'FAILED':<12} {'N/A':<10} ERROR")

print("-" * 60)

# Memory usage estimation
print("\n3. Memory Usage Analysis:")
total_size = sum(size for _, size in perf_files)
base64_overhead = total_size * 1.33  # Base64 encoding overhead
print(f"  Total file size: {total_size:,} bytes")
print(f"  Base64 encoded size: {base64_overhead:,.0f} bytes")
print(f"  Memory overhead: {(base64_overhead - total_size):,.0f} bytes ({((base64_overhead/total_size-1)*100):.1f}%)")

## Troubleshooting Guide

Common issues and solutions for the attachment system.

In [ ]:
print("=== TROUBLESHOOTING GUIDE ===")

# Test common error scenarios
print("\n1. Common Error Scenarios:")

# Test 1: Non-existent file
print("\nTest 1: Non-existent file")
bad_attachment = attachment_manager.attach_file(
    file_system=alice_fs,
    folder_name="documents",
    file_name="nonexistent.txt"
)
print(f"Result: {'SUCCESS' if bad_attachment else 'FAILED (Expected)'}")

# Test 2: Non-existent folder
print("\nTest 2: Non-existent folder")
bad_folder_attachment = attachment_manager.attach_file(
    file_system=alice_fs,
    folder_name="nonexistent_folder",
    file_name="quarterly_report.pdf"
)
print(f"Result: {'SUCCESS' if bad_folder_attachment else 'FAILED (Expected)'}")

# Test 3: Null file system
print("\nTest 3: Null file system")
null_fs_attachment = attachment_manager.attach_file(
    file_system=None,
    folder_name="documents",
    file_name="quarterly_report.pdf"
)
print(f"Result: {'SUCCESS' if null_fs_attachment else 'FAILED (Expected)'}")

# Test 4: Policy violations
print("\nTest 4: Policy violations")
strict_policy = AttachmentPolicy(
    max_attachment_size=100,  # Very small limit
    blocked_extensions=["pdf"],
    scan_for_malware=True
)

if 'report_attachment' in locals() and report_attachment:
    is_valid, error = attachment_manager.validate_attachment(report_attachment, strict_policy)
    print(f"PDF validation with strict policy: {'PASS' if is_valid else 'FAIL'}")
    if not is_valid:
        print(f"  Error: {error}")

print("\n2. Diagnostic Information:")
print(f"  AttachmentManager state: {attachment_manager.describe_state()}")
print(f"  Network nodes: {len(network.nodes)}")
print(f"  SMTP server status: {'Running' if smtp_server.operating_state else 'Stopped'}")
print(f"  POP3 server status: {'Running' if pop3_server.operating_state else 'Stopped'}")

print("\n3. Best Practices:")
print("  ✓ Always validate file system and folder existence before attaching")
print("  ✓ Check attachment policy compliance before sending")
print("  ✓ Scan attachments for malware before processing")
print("  ✓ Handle extraction errors gracefully")
print("  ✓ Log security violations for audit trails")
print("  ✓ Monitor attachment sizes to prevent memory issues")
print("  ✓ Use appropriate file health status for simulation accuracy")

print("\n4. Common Issues and Solutions:")
print("  Issue: 'File not found' errors")
print("    Solution: Verify folder and file names, check file system state")
print("  Issue: Policy validation failures")
print("    Solution: Verify folder and file names, check file system state")
print("Solution: Review attachment policy settings, check file sizes and extensions")
print("  Issue: Malware scan false positives")
print("    Solution: Verify file health status, check MIME type classification")
print("  Issue: Extraction failures")
print("    Solution: Ensure destination folder exists, check file permissions")

## Summary and Cleanup

Summary of demonstration and cleanup of resources.

In [ ]:
print("=== DEMONSTRATION SUMMARY ===")

# Collect statistics
total_files_created = sum(len(files) for files in test_files.values()) if 'test_files' in locals() else 0
total_attachments_processed = len(test_attachments) if 'test_attachments' in locals() else 0

print(f"\n📊 Statistics:")
print(f"  Files created: {total_files_created}")
print(f"  Attachments processed: {total_attachments_processed}")
print(f"  Network nodes: {len(network.nodes)}")
print(f"  Email clients: {len(email_clients)}")
print(f"  Mailboxes created: {len(users)}")

print(f"\n🎯 Key Features Demonstrated:")
print(f"  ✓ File attachment creation and extraction")
print(f"  ✓ Policy enforcement and validation")
print(f"  ✓ Malware scanning and threat detection")
print(f"  ✓ Multi-agent security scenarios")
print(f"  ✓ Performance testing and benchmarking")
print(f"  ✓ Error handling and troubleshooting")

print(f"\n🔒 Security Features:")
print(f"  ✓ File size and count limits")
print(f"  ✓ Extension-based filtering")
print(f"  ✓ Health status preservation")
print(f"  ✓ Malware detection (CORRUPT files)")
print(f"  ✓ Suspicious file classification")
print(f"  ✓ Double extension attack detection")
print(f"  ✓ Security violation logging")

print(f"\n📚 Next Steps:")
print(f"  1. Explore agent-based attachment scenarios")
print(f"  2. Implement custom attachment policies")
print(f"  3. Integrate with game scenarios")
print(f"  4. Test with larger file sizes")
print(f"  5. Develop custom malware detection rules")

# Cleanup
print(f"\n🧹 Cleanup:")
print(f"  Stopping email servers...")
if 'smtp_server' in locals() and smtp_server.operating_state.name == 'RUNNING':
    smtp_server.stop()
    print(f"  ✓ SMTP server stopped")

if 'pop3_server' in locals() and pop3_server.operating_state.name == 'RUNNING':
    print(f"  ✓ POP3 server stopped")

print(f"\n✅ Email attachment demonstration completed successfully!")

In [ ]:
# Returning to the original developer mode configuration.
PRIMAITE_CONFIG["developer_mode"]["enabled"] = original_dev_mode
PRIMAITE_CONFIG["developer_mode"]["output_sys_logs"] = original_sys_logs
PRIMAITE_CONFIG["developer_mode"]["output_agent_logs"] = original_agent_logs